In [1]:
import pandas as pd
import os
import sys
from datetime import datetime
import csv
import concurrent.futures
import subprocess
from datetime import datetime

Dataset of work "SZZ in the time of Pull Requests" 
- SOURCE: https://github.com/mozilla/regressors-regressions-dataset

In [2]:
# Clone Repository
# !hg clone https://hg.mozilla.org/mozilla-central/

In [ ]:
# Download dataset
# wget https://raw.githubusercontent.com/mozilla/regressors-regressions-dataset/main/dataset.csv

In [3]:
def getRevisionId(commit_hash):
    return subprocess.getoutput("cd firefox-source && hg id -n -r "+commit_hash)

In [4]:
def calculateDistanceCommits(bfc,bic):
    aux= bic + "::" + bfc
    distance = subprocess.getoutput("cd firefox-source && hg log --rev "+aux+" --template \"{rev} \" | wc -w")
    return int(distance)

In [5]:
def getDate(c_id):
    date = subprocess.getoutput("cd firefox-source && hg log -r "+c_id+" --template \"{date|date}\n\"")
    return datetime.strptime(date, "%a %b %d %H:%M:%S %Y %z")
def calculateDistanceDays(bfc_id,bic_id):
    bfc_date = getDate(bfc_id)
    bic_date = getDate(bic_id)
    return (bfc_date-bic_date).days

In [6]:
def addDistances(link):
    bfc = link['FIX_COMMITS_MERCURIAL'].split(" ")[0]
    bfc_id = getRevisionId(bfc)
    if isinstance(link['BUG_COMMITS_MERCURIAL'], str):
        bic = link['BUG_COMMITS_MERCURIAL'].split(" ")[0]
        bic_id = getRevisionId(bic)
    else:
        link['CALCULABLE_DISTANCE'] = False
        return
    link['DISTANCE_COMMITS'] = calculateDistanceCommits(bfc, bic)
    link['DISTANCE_DAYS'] = calculateDistanceDays(bfc_id,bic_id)
    link['CALCULABLE_DISTANCE'] = True

In [26]:
df = pd.read_csv('dataset.csv')
links = df[df['UNLINKABLE']==False].to_dict('records')
#links = df.to_dict('records')
len(links)

4174

In [27]:
future_results = []
with concurrent.futures.ThreadPoolExecutor(64) as executor:
    for link in links:
        future = executor.submit(addDistances, link)
        future_results.append(future)
for future in future_results:
    try:
        future.result() 
    except Exception as e:
        print(e)

In [28]:
df = pd.DataFrame.from_records(links)
df[['DISTANCE_COMMITS','DISTANCE_DAYS']].describe().T

,count,mean,std,min,25%,50%,75%,max
DISTANCE_COMMITS,4174.0,13881.897221,37878.281626,0.0,714.0,2604.5,10937.75,514394.0
DISTANCE_DAYS,4174.0,92.101342,269.759532,-933.0,5.0,18.0,72.00,4668.0


In [31]:
# There are 70 commits (from 4174) commits where BFC was created BEFORE BIC (???)
#df[df['DISTANCE_DAYS'] >= 0]
df[df['FIX_ID']==1345104]

,FIX_ID,FIX_COMMITS_MERCURIAL,FIX_COMMITS_GIT,BUG_ID,BUG_COMMITS_MERCURIAL,BUG_COMMITS_GIT,NO_FILE_SHARED_FILES,NEW_LINES_ONLY_FIX,REMOVE_LINES_ONLY_BUG,NO_BUG,UNLINKABLE,DISTANCE_COMMITS,DISTANCE_DAYS,CALCULABLE_DISTANCE
12,1345104,29103bace2abbd31d1e27d738483f6db26d07808,bb0f2387714608e8464906c308ff7f1be8c98423,1556437,dfdb610b82b95b7b64c5aa9a96c9e16bfacdf79b,196fec0c9cfb49841ec83f6e792f6bb9cdcded2b,False,False,False,False,False,0,-84,True


In [23]:
df[df['DISTANCE_DAYS'] >= 0 & df['CALCULABLE_DISTANCE']][['DISTANCE_COMMITS','DISTANCE_DAYS']].describe().T

,count,mean,std,min,25%,50%,75%,max
DISTANCE_COMMITS,4104.0,14118.278265,38156.376317,0.0,769.0,2742.5,11158.75,514394.0
DISTANCE_DAYS,4104.0,95.172027,270.068372,0.0,5.0,19.0,75.00,4668.0
